In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=5)

In [4]:
# Simple persistant state 
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]


In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        # New: Add a checkpointer to the graph
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")

#New: add memory
# abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
messages = [HumanMessage(content="What is the weather in zurich?")]
thread = {"configurable": {"thread_id": "1"}}

from langgraph.checkpoint.sqlite import SqliteSaver
checkpointer = SqliteSaver.from_conn_string(":memory:")
with checkpointer as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            # The message is printed as and when the persistence storage is updated
            print(v['messages'][0].content)
            
    # Ask a different message without contxt
    messages = [HumanMessage(content="What about in Berlin?")]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'][0].content)
            
    # Ask a question based on previous results
    messages = [HumanMessage(content="Which is warmer?")]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'][0].content)


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Zurich October 2023'}, 'id': 'call_Awls5Pjlj0L6Z5J6FoyiiBbP', 'type': 'tool_call'}
Back to the model!
[{'title': 'Weather in Zurich', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Zurich', 'region': '', 'country': 'Switzerland', 'lat': 47.3667, 'lon': 8.55, 'tz_id': 'Europe/Zurich', 'localtime_epoch': 1744543545, 'localtime': '2025-04-13 13:25'}, 'current': {'last_updated_epoch': 1744542900, 'last_updated': '2025-04-13 13:15', 'temp_c': 14.3, 'temp_f': 57.7, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.05, 'precip_in': 0.0, 'humidity': 77, 'cloud': 75, 'feelslike_c': 14.0, 'feelslike_f': 57.1, 'windchill_c': 16.3, 'windchill_f': 61.3, 'heatindex_c': 16.3, 'heatindex_

# Async Streaming

In [8]:
messages = [HumanMessage(content="What is the weather in zurich?")]
thread = {"configurable": {"thread_id": "5"}}

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
checkpointer = AsyncSqliteSaver.from_conn_string(":memory:")
async with checkpointer as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    async for event in abot.graph.astream_events({"messages": messages}, thread):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Zurich'}, 'id': 'call_XQjeeIknSBMMjULmI7vaDTG9', 'type': 'tool_call'}
Back to the model!
Currently| in| Zurich|,| the| weather| is| partly| cloudy| with| a| temperature| of| about| |14|.|3|°C| (|57|.|7|°F|).| The| wind| is| blowing| from| the| west| at| a| speed| of| |8|.|6| k|ph| (|5|.|4| mph|),| and| the| humidity| level| is| around| |77|%.| There| is| a| slight| chance| of| precipitation|,| with| a| recorded| |0|.|05| mm| of| rain|.|